In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt
import torchvision
from collections import defaultdict
import pickle
from torch.autograd import Variable
import torch.optim as optim
%matplotlib inline

In [ ]:
train_foldername = 'en-valid-10k'
train_filename = 'qa1_train.txt'
train_fname = './bAbI_Data/'+str(train_foldername)+'/'+str(train_filename)

valid_foldername = 'en-valid-10k'
valid_filename = 'qa1_valid.txt'
valid_fname = './bAbI_Data/'+str(valid_foldername)+'/'+str(valid_filename)

In [ ]:
train_dat_aux = []
valid_dat_aux = []
punctuations = ['.',',','?']

for l in open(train_fname):
    temp = ''.join(ch for ch in l if ch not in punctuations)
    train_dat_aux.append(temp.strip().split())
    
for l in open(valid_fname):
    temp = ''.join(ch for ch in l if ch not in punctuations)
    valid_dat_aux.append(temp.strip().split())

print(len(train_dat_aux))
print(len(valid_dat_aux))

In [ ]:
def hasDigits(input_str):
    return any(char.isdigit() for char in input_str)

In [ ]:
def create_vocab(data,unk_thres=0):
    aux = defaultdict(int)
    for i in range(len(data)):
        for j in range(1,len(data[i])):
            if hasDigits(data[i][j]):
                break
            aux[data[i][j]] += 1
    vocab = []
    unk_list = []
    for i in aux:
        if aux[i] < unk_thres:
            if not unk_list:
                vocab.append('UNK')
            unk_list.append(i)
        else:
            vocab.append(i)
    
    return vocab, unk_list

In [ ]:
def create_dictionaries(vocab):
    word2idx = defaultdict(int)
    idx2word = defaultdict(int)
    k = 0
    for i in range(len(vocab)):
        word2idx[vocab[i]] = k
        idx2word[k] = vocab[i]
        k += 1
    
    with open('variables/word2idx','wb') as handle:
        pickle.dump(word2idx,handle,protocol=pickle.HIGHEST_PROTOCOL)
    
    with open('variables/idx2word','wb') as handle:
        pickle.dump(idx2word,handle,protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
vocab, unk_list = create_vocab(train_dat_aux,0)
create_dictionaries(vocab)

In [ ]:
with open('variables/word2idx','rb') as handle:
    word2idx = pickle.load(handle)

with open('variables/idx2word','rb') as handle:
    idx2word = pickle.load(handle)

In [ ]:
def transform_data_BOW(data,vocab,unk_list,word2idx):
    N = len(vocab)
    dat_trans = np.zeros((len(data),N+1))
    for i in range(len(data)):
        if hasDigits(data[i][-1]):
            dat_trans[i,N] = word2idx[data[i][-2]]
            for j in range(1,len(data[i])-2):
                if data[i][j] in unk_list:
                    dat_trans[i,word2idx['UNK']] += 1
                else:
                    dat_trans[i,word2idx[data[i][j]]] += 1
        else:
            if data[i][0] == '1':
                dat_trans[i,N] = -2
            else:
                dat_trans[i,N] = -1
            for j in range(1,len(data[i])):
                #print(data[i][j],unk_list)
                if data[i][j] in unk_list:
                    dat_trans[i,word2idx['UNK']] += 1
                else:
                    dat_trans[i,word2idx[data[i][j]]] += 1
    
    return dat_trans

In [ ]:
train_data = transform_data_BOW(train_dat_aux,vocab,unk_list,word2idx)

In [ ]:
def smax(x):
    y = torch.div(torch.exp(x),torch.sum(torch.exp(x)))
    return y

In [ ]:
def comp(out,target):
    if (target.data[0] == np.argmax(smax(out.data))):
        return 1
    else:
        return 0

In [ ]:
class QuesAnsModel(torch.nn.Module):
    def __init__(self,embedding_dim, vocab_size, num_hops = 1, max_mem_size=15):
        super(QuesAnsModel,self).__init__()
        self.max_mem_size = max_mem_size
        self.vocab_size = vocab_size
        self.num_hops = num_hops
        self.embedding_dim = embedding_dim
        self.memory = Variable(torch.zeros((max_mem_size, vocab_size)).float())
        self.current_mem_size = 0
        self.embedding_A = torch.nn.Parameter(torch.randn(self.vocab_size,self.embedding_dim).float())
        self.embedding_B = torch.nn.Parameter(torch.randn(self.vocab_size,self.embedding_dim).float())
        self.embedding_C = torch.nn.Parameter(torch.randn(self.vocab_size,self.embedding_dim).float())
        self.W = torch.nn.Parameter(torch.randn(self.vocab_size,self.embedding_dim).float())
        self.softmax = torch.nn.Softmax()
    def forward(self, seq, tag):
        if tag == 's':
            if self.curr_mem_size < self.max_mem_size:
                self.memory[self.curr_mem_size] = Variable(torch.from_numpy(seq).float()).view(1,-1)
                self.curr_mem_size+=1
            else:
                pass
            return True
        elif tag == 'f':
            del self.memory
            self.curr_mem_size=0
            self.memory = Variable(torch.from_numpy(np.zeros((self.max_mem_size, self.vocab_size))).float())
            self.memory[0] = Variable(torch.from_numpy(seq).float()).view(1,-1)
            return True
        elif tag == 'q':
            self.question = Variable(torch.from_numpy(seq).float()).view(1,-1)
            ques_d = torch.mm(self.question,self.embedding_B)
            current_A = torch.mm(self.memory, self.embedding_A)
            current_C = torch.mm(self.memory, self.embedding_C)
            for i in range(self.num_hops):
                P = self.softmax(torch.mm(ques_d, current_A.t()).t())
                o = torch.mm(P.t(),current_C) + ques_d
                ques_d = o
            output = torch.mm(o, self.W.t())
            return output

In [ ]:
def train(model,tr_dt,epochs=10,eta=0.0001):
#     tr_dt = torch.from_numpy(np.array(tr_data))
    optimizer = optim.Adam(model.parameters(),lr=eta)
    loss = torch.nn.CrossEntropyLoss()
    tr_shape = tr_dt.shape
    eps = []
    l_tr = []
    accuracy = []
    for epoch in range(epochs):
        count=0;
        n_corr = 0;
        for i in range(tr_shape[0]):
            l_temp = 0
            tag = 'q'
            if(tr_dt[i,-1]==-1):
                tag = 's'
                model(tr_dt[i,:-1],tag)
            elif(tr_dt[i,-1]==-2):
                tag = 'f'
                model(tr_dt[i,:-1],tag)
            else:
                count+=1
                out = model(tr_dt[i,:-1],tag)
#                 print(out)
                target = Variable(torch.from_numpy(np.array([tr_dt[i,-1]])).type(torch.LongTensor))
                optimizer.zero_grad()
                loss_tr = loss(out,target)
                loss_tr.backward(retain_graph=True)
                optimizer.step()
                l_temp += loss_tr.data[0]
#                 if i % 15 == 14:
#                     print('[%d, %5d] loss: %.3f' % (epoch+1,i+1,l_temp/15))
#                 print('sds', out, target)
                n_corr += comp(out,target)
        acc = n_corr/count*100
        l_tr.append(l_temp)
        eps.append(epoch)
        accuracy.append(acc)
        print(epoch,'Loss : ',l_tr[-1],' , Acc : ',accuracy[-1])
        
    plt.plot(eps,l_tr)
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend(['Training Loss'])
    plt.savefig('Loss1.png')
    plt.show()

    plt.plot(eps,accuracy)
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy%')
    plt.legend(['Training Accuracy'],loc=4)
    plt.savefig('Acc2.png')
    plt.show()
    return l_tr, accuracy

In [ ]:
embedding_dim = 50
vocab_size = len(vocab)
num_hops = 2
max_mem_size = 15
epochs = 100
model = QuesAnsModel(embedding_dim, vocab_size, num_hops = num_hops, max_mem_size = max_mem_size)
train(model, train_data, epochs=epochs)